# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [1]:
import glob
import os

import btrack
import napari

import numpy as np

from skimage.io import imread

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
PATH = "/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/"

In [3]:
files = glob.glob(os.path.join(PATH, 'tracking', 'A1','field_1', 'masks','mask*.tif'))

# sort the files numerically
#files = sorted(files, key=lambda f: int(f[len(os.path.join(PATH, 'labels_')):-4]))
files = sorted(files)
phase_stack_filename = glob.glob(os.path.join(PATH, "AU02001_P_A1_1_reg_stack.tif"))[0]
phase_stack = imread(phase_stack_filename)

In [4]:
files
#phase_stack_filename

['/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask000.tif',
 '/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask001.tif',
 '/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask002.tif',
 '/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask003.tif',
 '/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask004.tif',
 '/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask005.tif',
 '/Users/dane/Documents/CellTrackingProjects/AU565/images/AU02001/Analysis/PC/intermediate_files/tracking/A1/field_1/masks/mask006.tif',
 '/Users/dane/Documents/CellTrackingProje

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 10 timepoints

In [5]:
def segmentation_arr(files):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in files:
        img = imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

In [6]:
stack = segmentation_arr(files)

Now we print out the shape of the stack (T, Z, Y, X):

In [7]:
stack.shape

(193, 1040, 1408)

### method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format. Note that the generator produces a numpy array for each image...

In [8]:
def segmentation_generator(files):
    """Segmentation generator"""
    
    for filename in files:
        img = imread(filename)
        yield img

In [9]:
generator = segmentation_generator(files)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x). Note that we can also use scikit-image `regionprops` to calculate properties for each object, using the `properties` keyword:

obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=('area', ))

# inspect the first object
obj_from_arr[0]

In [10]:
obj_from_generator = btrack.utils.segmentation_to_objects(
    generator, 
    properties = ('area', 'major_axis_length')
)

In [11]:
# inspect the first object
obj_from_generator[0]

,ID,x,y,z,t,dummy,states,label,prob,area,major_axis_length
0,0,21.118557,6.762887,0.0,0,False,0,5,0.0,194,16.759275


## run btrack with the objects

We will use the objects from the generator here.

In [12]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config_AU565.json')
    tracker.max_search_radius = 100

    # append the objects to be tracked
    tracker.append(obj_from_generator)

    # set the volume
    tracker.volume=((0, 1408), (0, 1040), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=200)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)
    
    #tracks = tracker.tracks

[INFO][2022/05/02 08:40:18 AM] btrack (v0.4.3) library imported
[INFO][2022/05/02 08:40:18 AM] Setting max XYZ search radius to: 100
[INFO][2022/05/02 08:40:18 AM] Starting BayesianTracker session
Configuration file ../models/cell_config_AU565.json not found
[INFO][2022/05/02 08:40:18 AM] Ending BayesianTracker session


OSError: Configuration file ../models/cell_config_AU565.json not found

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [14]:
#viewer = napari.Viewer()
#viewer = napari.view_image(img_p_stack, name = "phase image")

viewer = napari.view_image(phase_stack, name = "phase image")
viewer.add_labels(stack,  name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')

napari.run()

In [15]:
graph

{295: [158],
 296: [169],
 297: [169],
 311: [39],
 313: [39],
 324: [110],
 326: [110],
 327: [102],
 343: [158],
 355: [222],
 361: [17],
 363: [17],
 375: [115],
 378: [115],
 381: [102],
 388: [136],
 389: [136],
 395: [185],
 397: [185],
 406: [222],
 481: [46],
 482: [46],
 566: [406],
 567: [406],
 581: [358],
 591: [579],
 593: [358],
 609: [579],
 624: [576],
 625: [576],
 640: [625],
 641: [625],
 654: [286],
 655: [286],
 687: [634],
 688: [634],
 690: [375],
 695: [640],
 699: [640],
 722: [375],
 781: [772],
 782: [772],
 801: [148],
 804: [148],
 836: [795],
 838: [795],
 852: [674],
 853: [674],
 857: [825],
 871: [825],
 896: [801],
 898: [801],
 921: [876],
 923: [876],
 1013: [60],
 1015: [60],
 1018: [122],
 1037: [122],
 1039: [627],
 1040: [627],
 1051: [367],
 1052: [367],
 1074: [932],
 1075: [932],
 1118: [1103],
 1120: [908],
 1121: [908],
 1130: [1083],
 1140: [1103],
 1143: [1083],
 1178: [348],
 1197: [348],
 1212: [1029],
 1213: [1029],
 1279: [256],
 1281:

In [101]:
graph

{295: [158],
 296: [169],
 297: [169],
 311: [39],
 313: [39],
 324: [110],
 326: [110],
 327: [102],
 343: [158],
 355: [222],
 361: [17],
 363: [17],
 375: [115],
 378: [115],
 381: [102],
 388: [136],
 389: [136],
 395: [185],
 397: [185],
 406: [222],
 481: [46],
 482: [46],
 566: [406],
 567: [406],
 581: [358],
 591: [579],
 593: [358],
 609: [579],
 624: [576],
 625: [576],
 640: [625],
 641: [625],
 654: [286],
 655: [286],
 687: [634],
 688: [634],
 690: [375],
 695: [640],
 699: [640],
 722: [375],
 781: [772],
 782: [772],
 801: [148],
 804: [148],
 836: [795],
 838: [795],
 852: [674],
 853: [674],
 857: [825],
 871: [825],
 896: [801],
 898: [801],
 921: [876],
 923: [876],
 1013: [60],
 1015: [60],
 1018: [122],
 1037: [122],
 1039: [627],
 1040: [627],
 1051: [367],
 1052: [367],
 1074: [932],
 1075: [932],
 1118: [1103],
 1120: [908],
 1121: [908],
 1130: [1083],
 1140: [1103],
 1143: [1083],
 1178: [348],
 1197: [348],
 1212: [1029],
 1213: [1029],
 1279: [256],
 1281: